In [ ]:
!pip install datasets
!pip install transformers
!pip install pinecone-client
!pip install sentence_transformers
!pip install apache_beam

In [16]:
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd
import pinecone
import datasets
from sentence_transformers import SentenceTransformer
import torch
import apache_beam

In [12]:
device = torch.device("cuda")

In [19]:
API_KEY = "4da56e04-60e9-4e45-90ae-c9cba9b01686"
ENV = "us-west1-gcp-free"

wiki_data = load_dataset(
    'wiki_snippets',
    'wiki40b_en_100_0',
    split='train',
    streaming=True
).shuffle(seed=25)

In [ ]:
city = wiki_data.filter(lambda d: 'Bologna' in d['section_title'])

total_doc_count = 5000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(city, total=total_doc_count):
    # extract the fields we need
    doc = {
        "article_title": d["article_title"],
        "section_title": d["section_title"],
        "passage_text": d["passage_text"]
    }
    # add the dict containing fields we need to docs list
    docs.append(doc)

    # stop iteration once we reach 50k
    if counter == total_doc_count:
        break

    # increase the counter on every iteration
    counter += 1

df = pd.DataFrame(docs)

pinecone.init(api_key=API_KEY, environment=ENV)

index_name = "abstractive-question-answering-bologna" e)

# check if the abstractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=retriever.get_sentence_embedding_dimension(),
        metric="cosine"
    )

# connect to abstractive-question-answering index we created
index = pinecone.Index(index_name)

batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch["passage_text"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/5000 [00:00<?, ?it/s]